<a href="https://colab.research.google.com/github/nancyfosua/NLP_Assignment/blob/main/n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [3]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [4]:

print("load training set..")
print("\n")
train_data, vocab = load_data("/content/drive/MyDrive/Colab Notebooks/NLP/week1/train1 (1).txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("/content/drive/MyDrive/Colab Notebooks/NLP/week1/valid1 (1).txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [5]:
# data = [
#         ["dog",'chief'],['yaw', 'iwot'],['pot','dance']]
# for i in range(len(data)):
#   for j in range(len(data[i])):
#     if data[i][j] == 'dog':
#       data[i][j]= 'nancy'

# print(data)

In [6]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    for text in data:
      text_with_unk = []
      for word in text:
        if vocab[word] > mincount and word in vocab:
          text_with_unk.append(word)
        else:
          text_with_unk.append('<unk>')
      data_with_unk.append(text_with_unk)


    return data_with_unk

In [7]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [8]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    
    print(tuple(data[0]))
    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value

       
        for i in range(len(sentence)):
            for j in range(n):
                if i-j >= 0 :
                    counts[sentence[i-j:i]][sentence[i]] += 1
                else:
                  break
                   
    # print(counts)
    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys(): #
    ## FILL CODE
      # print(context)
        d = 0
        for word in counts[context].keys():  
            d += counts[context][word]
        # print(word)
        for word in counts[context].keys():
            prob[context][word] = counts[context][word] / d 

    return prob

In [9]:
# RUN TO BUILD NGRAM MODEL
n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  2
('<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>')


In [10]:
model

defaultdict(<function __main__.build_ngram.<locals>.<lambda>>,
            {(): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
                         {'<s>': 0.08427417521633049,
                          'my': 0.00307636352614909,
                          '<unk>': 0.08683528573809993,
                          "don't": 0.003015745525633837,
                          'speak': 0.000181854001545759,
                          '</s>': 0.08427417521633049,
                          'tomorrow': 6.0618000515253e-05,
                          'is': 0.01113855759467774,
                          'the': 0.07075636110142906,
                          'election.': 4.546350038643975e-05,
                          'he': 0.006516435055389698,
                          'worked': 7.577250064406626e-05,
                          'hard': 0.000242472002061012,
                          'to': 0.026687074726840135,
                          'make': 0.000666798005667783,


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [11]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''
    

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model

    if model[context][w] > 0:
      prob = model[context][w] 
      return prob
    else:
      prob = 0.4* get_prob(model, context[1:], w)
      return prob


In [12]:
get_prob(model, tuple(valid_data[0][:5]) ,valid_data[0][5]) 

5.4829727993146295e-06

In [13]:
  def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    p = 0.0
    t = 0
    
    for text in data:
      text = tuple(text)
      for i in range(len(text)):
          context = text[i - min(n-1, i): i]
          p += np.log(get_prob(model, context, text[i])) 
          t += 1
    return np.exp(- p /t )

In [14]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 47.5824831044242


In [15]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    if context in model:
      return model[context]
    else:
      return get_proba_distrib(model, context[1:])

In [16]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        prob_sent_gen = get_proba_distrib(model, tuple(sentence))
        gene_word = np.random.choice(list(prob_sent_gen.keys()), 1, p = list(prob_sent_gen.values()))
        sentence.append(gene_word[0])
    return sentence

In [17]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', "they're", '<unk>', 'and', 'take', 'care', 'whether', 'mary', 'that', 'on', 'the', 'locked', 'door.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.